In [39]:
from keras.applications import vgg16
from keras.layers import Dense, Flatten
from keras.models import Model

# n_chan = 2
# model = vgg16.VGG16(input_shape=[201, 201, n_chan], classes=17, include_top=True, weights=None)

n_chan = 3
model_pre = vgg16.VGG16(input_shape=[201, 201, n_chan], classes=1000, include_top=False, weights='imagenet')
layer = model_pre.output
layer = Flatten()(layer)
layer = Dense(4096, activation="relu")(layer)
layer = Dense(4096, activation="relu")(layer)
new_output = Dense(17, activation="softmax")(layer)
model = Model(inputs=model_pre.input, outputs=new_output)
model.summary()

In [40]:
from keras import backend as K

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        K.print_tensor(y_true)
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [41]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', metrics=[f1], optimizer=Adam(lr=1e-3))

In [42]:
from dataGenerator import DataGenerator, TestDataGenerator
from personal_data import location_data
dataGenerator_train = DataGenerator(location_data, n_channels=n_chan, batch_size=128)
dataGenerator_val = DataGenerator(location_data, n_channels=n_chan, batch_size=128, isTrainingData=False)
dataGenerator_test = TestDataGenerator('../validation', n_channels=n_chan, batch_size=128)

In [43]:
from keras import callbacks
import time

# tensorboard --logdir ~/cytodata2019/tensorboard &
tensorboard = callbacks.TensorBoard(update_freq='batch',
    log_dir='../tensorboard/' + time.strftime("%Y%m%d-%H%M%S"))

In [44]:
model.fit_generator(dataGenerator_train.generate(),
    validation_data=dataGenerator_val.generate(),
    steps_per_epoch=len(dataGenerator_train),
    validation_steps=len(dataGenerator_val),
    callbacks=[tensorboard], verbose=1, epochs=100)

Epoch 1/100


ValueError: Error when checking target: expected dense_4 to have 2 dimensions, but got array with shape (128, 1, 1, 17)

In [0]:
predictions = model.predict_generator(dataGenerator_test.generate(), steps=len(dataGenerator_test))

In [0]:
import numpy as np
hit_indices = np.argmax(predictions, axis=1).astype(dtype=np.int)
hit_names = [dataGenerator_train.labelsNames[i] for i in hit_indices]

In [0]:
cell_names = [s[26:-5] for s in dataGenerator_test.allData] 

In [0]:
print(len(cell_names), len(hit_names))
with open('output.csv', 'w') as file:
    file.write("cell_code,prediction\n")
    for (c,h) in zip(cell_names, hit_names):
        file.write(f"{c},{h}\n")
